# Language Features

## Attributes

In [ ]:
def streaming_average(n):
    streaming_average.count += 1
    streaming_average.total += n
    
    return streaming_average.total / streaming_average.count

streaming_average.count = 0
streaming_average.total = 0

In [ ]:
print(streaming_average(2))
print(streaming_average(3))
print(streaming_average(4))

## Closures in Python

In [ ]:
def make_streaming_average():
    count = 0
    total = 0

    def inner(n):
        nonlocal count, total

        count += 1
        total += n

        return total / count

    return inner

In [ ]:
streaming_average = make_streaming_average()
print(streaming_average(2))
print(streaming_average(3))
print(streaming_average(4))

## Sales tax calculator

In [ ]:
def make_sales_tax_calculator(rate):
    def calculator(amount):
        return amount * rate
    
    return calculator

In [ ]:
vat_calculator = make_sales_tax_calculator(0.2)
print(vat_calculator(100))

new_york_sales_tax_calculator = make_sales_tax_calculator(0.04)
print(new_york_sales_tax_calculator(100))

## Closing on references

In [ ]:
def make_sales_tax_calculator(rates):
    def calculator(amount):
        return amount * rates["uk"]
    
    return calculator

In [ ]:
rates = {"uk" : 0.2}
vat_calculator = make_sales_tax_calculator(rates)
print(vat_calculator(100))
rates["uk"] = 0.05
print(vat_calculator(100))

In [ ]:
rates = {"uk" : 0.2}
vat_calculator = make_sales_tax_calculator(dict(rates))
print(vat_calculator(100))
rates["uk"] = 0.05
print(vat_calculator(100))

## Modifying closure state

In [ ]:
def make_sales_tax_calculator(rate):
    def calculator(amount):
        return amount * rate
    
    def change_rate(value):
        nonlocal rate
        rate = value
    
    calculator.change_rate = change_rate
    
    return calculator

In [ ]:
vat_calculator = make_sales_tax_calculator(0.2)
print(vat_calculator(100))
vat_calculator.change_rate(0.05)
print(vat_calculator(100))

## Context manager examples

In [ ]:
import pathlib

book_file_path = pathlib.Path("../data/dracula.txt")

with book_file_path.open() as f:
    book = [line.strip() for line in f.readlines()]
    
book[1876:1886]

In [ ]:
import json
import pathlib
from pprint import pprint as pp

document_path = pathlib.Path("../data/countries.json")

with document_path.open() as f:
    countries = json.load(f)
    
pp(countries[234])

In [ ]:
import pandas as pd
import pathlib

document_path = pathlib.Path("../data/countries.json")

countries_df = pd.read_json(document_path)
countries_df.set_index("alpha-3", inplace=True)

with pd.option_context("display.max_rows", 6), pd.option_context("display.max_columns", 4):
    print(countries_df)

## Custom context managers

In [ ]:
from contextlib import contextmanager
from time import perf_counter

@contextmanager
def timer_context_manager():
    start = perf_counter()
    end = 0.0
    
    yield lambda: end - start
    
    end = perf_counter()

In [ ]:
from time import sleep

with timer_context_manager() as timer:
    sleep(0.5)
    
print(timer())

## Class-based custom context managers

In [ ]:
from time import perf_counter

class Timer:
    def __enter__(self):
        self.start = perf_counter()
        self.end = 0.0
        
        return lambda: self.end - self.start

    def __exit__(self, *args):
        self.end = perf_counter()

In [ ]:
from time import sleep

with Timer() as timer:
    sleep(1.5)
    
print(timer())

## Handling exceptions in context managers

In [ ]:
import json

@contextmanager
def json_context_manager(document_path):
    try:
        f = open(document_path)
        document = json.load(f)
    except FileNotFoundError:
        f = None
        document = None
        
    yield document
        
    if f:
        f.close()

## Using JSON context manager

In [ ]:
from pprint import pprint as pp

with json_context_manager("../data/missing.json") as document:
    if document:
        pp(document[234])
    else:
        print("Document doesn't exist")

## Handling exceptions in class-based context managers

In [ ]:
import json

class JsonContextManager:
    def __init__(self, document_path):
        self.document_path = document_path
        
    def __enter__(self):
        try:
            self.f = open(self.document_path)
            document = json.load(self.f)
        except FileNotFoundError:
            self.f = None
            document = None
        
        return document

    def __exit__(self, *args):
        if self.f:
            self.f.close()

In [ ]:
from pprint import pprint as pp

with JsonContextManager("../data/missing.json") as document:
    if document:
        pp(document[234])
    else:
        print("Document doesn't exist")

## Decorators

In [ ]:
def halloween_decorator(func):
    def wrapper():
        print("Boo!")
        
        func()
    
    return wrapper

In [ ]:
@halloween_decorator
def say_hi():
    print("Hi.")

In [ ]:
say_hi()

## Decorating functions with arguments

In [ ]:
def halloween_decorator(func):
    def wrapper(*args, **kwargs):
        print("Boo!")
        
        func(*args, **kwargs)
    
    return wrapper

In [ ]:
@halloween_decorator
def say_hi(name):
    print(f"Hi, {name}.")

In [ ]:
say_hi("Andrew")

## Returning values from decorated functions

If we don't return the value of the decorated function from the decorator, the decorator will swallow it.

In [ ]:
def halloween_decorator(func):
    def wrapper(*args, **kwargs):
        print("Boo!")
        
        return func(*args, **kwargs)
    
    return wrapper

In [ ]:
@halloween_decorator
def say_hi(name):
    return f"Hi, {name}."

In [ ]:
greeting = say_hi("Andrew")
print("Greeting:", greeting)

## Decorators with arguments

In [ ]:
def halloween_decorator(exclamation):
    def decorator(func):
        def wrapper(*args, **kwargs):
            print(exclamation)

            func(*args, **kwargs)

        return wrapper
    
    return decorator

In [ ]:
@halloween_decorator("Woo...")
def say_hi():
    print("Hi.")

In [ ]:
say_hi()

## Preserving decorated function identities

In [ ]:
print(say_hi.__name__)

In [ ]:
import functools

def halloween_decorator(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print("Boo!")
        
        func(*args, **kwargs)
    
    return wrapper

In [ ]:
@halloween_decorator
def say_hi():
    return "Hi."

In [ ]:
print(say_hi.__name__)

## Stateful decorators

In [ ]:
def track_invocations_decorator(func):
    invocations = {}
    
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        count = invocations.get(func.__name__, 0) + 1
        invocations[func.__name__] = count
        
        print(f"{func.__name__} has been called {count} time(s)")
        
        func(*args, **kwargs)
        
    return wrapper

## Using a stateful decorator

In [ ]:
@track_invocations_decorator
def say_hi():
    print("Hi.")
    
@track_invocations_decorator
def scare_me():
    print("Boo!")

In [ ]:
say_hi()
scare_me()

## Debugging decorator

In [ ]:
import functools

def debug(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        args_representation = [repr(a) for a in args]
        kwargs_representation = [f"{k}={v!r}" for k, v in kwargs.items()]
        signature = ", ".join(args_representation + kwargs_representation)
        print(f"Calling {func.__name__}({signature})")
        
        value = func(*args, **kwargs)
        print(f"{func.__name__!r} returned {value!r}")
        
        return value
    
    return wrapper

## Using the debugging decorator

In [ ]:
@debug
def cube(n):
    return n**3

In [ ]:
print(cube(n=4))

In [ ]:
import math

factorial = debug(math.factorial)

print(factorial(6))

## Caching decorator

In [ ]:
import functools

def cache(func):
    cached_values = {}
    
    @functools.wraps(func)    
    def wrapper(*args, **kwargs):
        key = args + tuple(kwargs.items())
        if key not in cached_values:
            cached_values[key] = func(*args, **kwargs)
        
        return cached_values[key]
        
    return wrapper

In [ ]:
import math

@cache
def is_prime(n):
    if n <= 1:
        return False

    for i in range(2, int(math.sqrt(n)) + 1):
        if n % i == 0:
            return False

    return True

In [ ]:
from timeit import timeit

print(timeit(lambda: is_prime(270001008058013), number=1))
print(timeit(lambda: is_prime(270001008058013), number=1))

## Monkey patching functions

In [ ]:
import numpy as np

def estimate_project_duration(min, max):
    return np.random.uniform(min, max)

In [ ]:
print(estimate_project_duration(5, 10))

In [ ]:
original_function = np.random.uniform

In [ ]:
np.random.uniform = lambda low, high: np.random.triangular(
    low, (low + high) / 2, high
)

In [ ]:
print(estimate_project_duration(5, 10))

In [ ]:
np.random.uniform = original_function

## Dynamically creating classes

In [ ]:
import math

def init_circle(self, radius):
    self.radius = radius

Circle = type("Circle", (), {
    "__init__": init_circle,
    "calculate_area": lambda x: math.pi * x.radius ** 2
})

circle = Circle(5)

print(circle.calculate_area())

## Plug-in registration

In [ ]:
class PlugInRegistry(type):
    plug_ins = []
    def __init__(cls, name, bases, attrs):
        if name != "PlugIn":
            PlugInRegistry.plug_ins.append(cls)

class PlugIn(metaclass=PlugInRegistry):
    pass

class ImagePlugIn(PlugIn):
    pass

class AudioPlugIn(PlugIn):
    pass

class VideoPlugIn(PlugIn):
    pass

print(PlugInRegistry.plug_ins)

## Dataclasses

In [ ]:
from dataclasses import dataclass

@dataclass
class Wager:
    selection: str
    kind: str
    price: float
    
bet1 = Wager("Chelsea", "win", 5.)
bet2 = Wager("Chelsea", "win", 5.)

print(bet1)
print(bet1.selection)
print(bet1 == bet2)

In [ ]:
class WagerRegularClass:
    def __init__(self, selection, kind, price):
        self.selection = selection
        self.kind = kind
        self.price = price
    
bet1_rc = WagerRegularClass("Chelsea", "win", 5.)
bet2_rc = Wager("Chelsea", "win", 5.)

print(bet1_rc)
print(bet1_rc.selection)
print(bet1_rc == bet2_rc)

## Structural pattern matching

In [ ]:
file_type = "gif"

match file_type:
    case "gif":
        print("Process JPG file")   
    case "jpg":
        print("Process PNG file")
    case "png":
        print("Process JPG file")
    case _:
        print("Process unknown file")             

## Structural pattern matching with enums

In [ ]:
from enum import Enum

class FileType(Enum):
    GIF = 0
    JPG = 1
    PNG = 2
    
file_type = FileType.GIF

match file_type:
    case FileType.GIF:
        print("Process JPG file")   
    case FileType.GIF:
        print("Process PNG file")
    case FileType.GIF:
        print("Process JPG file")
    case _:
        print("Process unknown file")  

## Structural pattern matching with multiple elements

In [ ]:
fractional_price = [3, 0]

match fractional_price:
    case [n, 0]:
        print("Infinity")
    case [n, 1]:
        print(n)
    case [n, d]:
        print(n / d)
    case _:
        print("Not a fractional price)            

## Structural pattern matching with guards

In [ ]:
fractional_price = [3, -1]

match fractional_price:
    case [n, d] if (n < 0) or (d < 0):
        print("Cannot have negative prices")
    case [n, 0]:
        print("Infinity")
    case [n, 1]:
        print(n)
    case [n, d]:
        print(n / d)
    case _:
        print("Not a fractional price)       